In [1]:
from pathlib import Path
thispath = Path('__file__').resolve()
base_path = thispath.parent.parent
import sys; sys.path.insert(0, str(base_path))
import json
sys.path.append('../utils/')
import time

In [2]:
import SimpleITK as sitk
import pandas as pd
from tqdm.auto import tqdm
import time
import numpy as np
from metrics import get_landmarks_array_from_txt_file, target_registration_error
from elastic_helpers import elastix_wrapper, modify_field_parameter_map,transformix_wrapper

In [11]:
base_path = Path().resolve().parent
params_path = base_path / 'parameters'
output_path = base_path /'elastix'
results_path = base_path / 'experiments' / 'elastik' / 'results.json'


In [4]:
dataset = pd.read_csv('../result_data/dir_lab_copd.csv')

In [5]:
def registration(dataset,param_maps_to_use,experiment_name):
    results = {}

    for i in tqdm(range(len(dataset))):
        sample = dataset.iloc[i]
        sample_data = sample['case']

        pm = param_maps_to_use[-1]
        data_path = base_path / 'result_data' / sample_data
        output_pm_path = output_path / pm.rstrip('.txt') / experiment_name
        output_pm_path.mkdir(exist_ok=True, parents=True)
        parameters_filename = params_path
        result_path = output_path / sample['case'] / experiment_name
        result_path.mkdir(exist_ok=True, parents=True)
        param_maps_to_use_ = [str(params_path / p) for p in param_maps_to_use]
        name = sample['case']
        i_temp_path = data_path / f'{sample_data}_iBHCT.nii.gz'
        e_temp_path = data_path / f'{sample_data}_eBHCT.nii.gz'
        
        print('Registration of image...')
        start = time.time()
        transform_map_path = elastix_wrapper(
            i_temp_path, e_temp_path, result_path, param_maps_to_use_,
        )
        reg_time = time.time() - start
        name = f'{sample_data}_300_iBH_xyz_r1.txt'
        lm_points_filepath = data_path / name
        # Correct transformation parameters file
        field_value_pairs = [
            ('ResultImageFormat', 'nii.gz'),
            ('ResultImagePixelType', "float"),
            ('FinalBSplineInterpolationorder', '0')
        ]

        modify_field_parameter_map(field_value_pairs, transform_map_path)
        # Transform landmarks
        print('Transforming points...')
        lm_out_filepath = result_path.parent / f'r_{sample_data}'
        transformix_wrapper(
            lm_points_filepath, lm_out_filepath, transform_map_path,
            points=True)
                
        landmarks = get_landmarks_array_from_txt_file(lm_out_filepath / name)
        e_landmark_pts = pd.read_csv(sample['e_landmark_pts'], header=None, index_col=None).values
        m, s = target_registration_error(landmarks, e_landmark_pts, [sample['space_x'],sample['space_y'],sample['space_z']])
        print(f'{"-"*10} {sample_data} {"-"*10}')
        print(f'Provided displacement: 21.77 | 6.46 ')
        print(f'Computed displacement: {m} | {s}')
        
        results[sample['case']] = {
            'mean_tre': m,
            'std_tre': s,
            'time': reg_time,
        }
    
    return results

In [6]:
experiment_name = 'Par0054_sstvd'
param_maps_to_use = [
        'Par0054_sstvd.txt'
]

In [14]:
results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 21.03 | 10.12
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 18.76 | 6.0
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 7.59 | 4.84
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 24.55 | 9.36


In [13]:
print(results)

{'copd1': {'mean_tre': 21.03, 'std_tre': 10.12, 'time': 188.22110414505005}, 'copd2': {'mean_tre': 18.76, 'std_tre': 6.0, 'time': 162.82338285446167}, 'copd3': {'mean_tre': 7.59, 'std_tre': 4.84, 'time': 207.8776981830597}, 'copd4': {'mean_tre': 24.55, 'std_tre': 9.36, 'time': 188.10323524475098}, 'Par0054_sstvd': {'copd1': {'mean_tre': 21.03, 'std_tre': 10.12, 'time': 200.30659651756287}, 'copd2': {'mean_tre': 18.76, 'std_tre': 6.0, 'time': 178.25403809547424}, 'copd3': {'mean_tre': 7.59, 'std_tre': 4.84, 'time': 215.0891616344452}, 'copd4': {'mean_tre': 24.55, 'std_tre': 9.36, 'time': 206.71247172355652}}}
